## Setup

In [1]:
import numpy as np

from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import H, I
from random import randint
from math import ceil

## Define Functions

In [2]:
def y_func(x) :
    if x == "000":
        z = 1
    elif x == "100":
        z = 0
    elif x == "001":
        z = 2
    else: 
        z = 3
    return z

In [3]:
def dirac(x) :
    if x == 0:
        return 1
    else:
        return 0

## Define Quantum Oracle

In [4]:
STRING_A = "100"
STRING_B = "000"
STRING_C = "001"

N = len(STRING_A)

O_1 = np.identity(2 ** N)
O_2 = np.zeros(shape=(2 ** N, 2 ** N))
O_3 = np.zeros(shape=(2 ** N, 2 ** N))
O_4 = np.zeros(shape=(2 ** N, 2 ** N))
                   
for b in range(2 ** N):
    if np.binary_repr(b, N) == STRING_A:
        O_2[b, b] = -1
    else:
        O_2[b, b] = 1

for b in range(2 ** N):
    if np.binary_repr(b, N) == STRING_A or np.binary_repr(b, N) == STRING_B:
        O_3[b, b] = -1
    else:
        O_3[b, b] = 1
        
for b in range(2 ** N):
    if np.binary_repr(b, N) == STRING_A or np.binary_repr(b, N) == STRING_B or np.binary_repr(b, N) == STRING_C:
        O_4[b, b] = -1
    else:
        O_4[b, b] = 1

# x randomely chosen so that we can see the oracle works
x = np.binary_repr(randint(0, 2 ** N), N)
y = y_func(x)

oracle = O_1 * dirac(y - 0) + O_2 * dirac(y - 1) + O_3 * dirac(y - 2) + O_4 * dirac(y - 3)

print(x)
print(y)
print(oracle)

100
0
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


## Initialization

In [5]:
qvm = QVMConnection()
gr_prog = Program()
#d_prog = Program()

In [6]:
first = list(reversed(range(N)))
#gr_prog.inst([I(q) for q in first])
#second = list(reversed(range(2)))
#d_prog.inst([I(q) for q in second])
#print(qvm.wavefunction(gr_prog))
#print(qvm.wavefunction(d_prog))

## The Loop

In [7]:
# Define quantum oracle
ORACLE_GATE_NAME = "GROVER_ORACLE"
gr_prog.defgate(ORACLE_GATE_NAME, oracle)

# Define inversion around the mean
DIFFUSION_GATE_NAME = "DIFFUSION"
diffusion = 2.0 * np.full((2**N, 2**N), 1/(2**N)) - np.eye(2**N)
gr_prog.defgate(DIFFUSION_GATE_NAME, diffusion)

In [8]:
# Number of algorithm iterations
N_ITER = int(np.pi / 4 * np.sqrt(2**N))

# Set Starting Point
x = np.binary_repr(randint(0, 2 ** N), N)
y = y_func(x)
m = 1
lambd = 8/7

# loop
for i in range(N_ITER):
    
    # Check x, y and m
    #print(x)
    #print(y)
    #print(m)
    
    
    # Initialize
    gr_prog.inst([I(q) for q in first])
    
    # Apply H Gate
    gr_prog.inst([H(q) for q in first])
    
    # Set Quantum Oracle
    oracle = O_1 * dirac(y - 0) + O_2 * dirac(y - 1) + O_3 * dirac(y - 2) + O_4 * dirac(y - 3)
    gr_prog.defgate(ORACLE_GATE_NAME, oracle)
    #print(oracle)
    
    # Set r
    r = randint(1,m)
    
    
    #Apply Grover's Search
    for i in range(r):
        
        # \psi_2^i:  Apply Quantum Oracle
        gr_prog.inst(tuple([ORACLE_GATE_NAME] + first))
        #print(qvm.wavefunction(gr_prog))
    
        # \psi_3^i:  Apply Inversion around the mean
        gr_prog.inst(tuple([DIFFUSION_GATE_NAME] + first))
        #print(qvm.wavefunction(gr_prog))
    
    # Measure Output and set y
    ## Measure
    for q in first:
        gr_prog.measure(qubit_index=q, classical_reg=q)
        
    ## Run    
    ret = qvm.run(gr_prog, classical_addresses=first)
    ret_string = ''.join([str(q) for q in ret[0]])
    y_temp = y_func(ret_string) #conversion needs to be right
    #print(ret_string)
    #print(y_temp)
    
    #Set new threshold
    if y_temp < y:
        x = ret_string
        y = y_func(ret_string)
        m = ceil(lambd*m)


#print(x)
#print(y)

/Users/felixschildorfer/anaconda3/lib/python3.7/site-packages/pyquil/quil.py:135: UserWarning: Gate GROVER_ORACLE has already been defined in this program
  .format(instruction.name))


## Measurement

In [9]:
# Run
print("The searched input is: {}".format(x))
print("The minimum is: {}".format(y))

The searched input is: 100
The minimum is: 0


### Source: https://arxiv.org/pdf/1804.10068.pdf